**SVM Soft Margin Primal**

Alexandra Belinda Santosa - 
a1785180

In [1]:
#mount from google collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import modules needed
import pandas as pd
import numpy as np
import os
import math as m
import cvxopt
import cvxopt.solvers
from cvxopt import matrix
from cvxopt import solvers
from numpy import array

#sckitlearn.svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline
np.random.seed(40)

In [3]:
X_train, Y_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/train.csv', header=None), pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/test.csv',header=None)
X_data_train = X_train.iloc[:4000,1:].to_numpy()
X_data_val=X_train.iloc[4000:,1:].to_numpy()
y_label_train, y_label_val  = X_train.iloc[:4000,0].to_numpy(), X_train.iloc[4000:,0].to_numpy()
Y, y = Y_test.iloc[:,1:].to_numpy(), Y_test.iloc[:,0].to_numpy()
y_label_train, y_label_val, y = np.where(y_label_train == 0, -1, y_label_train), np.where(y_label_val == 0, -1, y_label_val), np.where(y == 0, -1, y)

Starting off with the SVM training primal function

In [4]:
def svm_train_primal(data_train, label_train, regularisation_para_C):
    c, d = data_train.shape
    #set up the array
    Pw = np.asarray(np.diag(np.ones(d)))
    Ps = np.zeros([d, c])
    Pb = np.zeros([1, d]) 
    P1=np.zeros([c, c])
    P2=np.zeros([c, 1])
    P3=np.zeros([1, c])
    P4=np.zeros([1, 1]) 

    #Set up the matrices
    P = cvxopt.matrix(np.concatenate((np.concatenate((Pw, Ps, Pb.T), axis = 1), np.concatenate((Ps.T, P1, P2), axis=1), 
                                      np.concatenate((Pb, P3, P4),axis = 1)), axis = 0))
    q = cvxopt.matrix((np.concatenate((Pb, np.matrix(np.ones([c]) * regularisation_para_C/c), P4), axis=1).T))

    
    g1= np.dot(np.diag(label_train), data_train) * (-1)
    g2= np.asarray(np.diag(np.ones([c]) * (-1)))
    g3= np.matrix(label_train) * (-1)
    g3 = g3.T
    G = np.vstack((np.concatenate((g1,g2,g3), axis = 1), np.concatenate((np.zeros([c, d + 1]), np.asarray(np.diag(np.ones([c]) * (-1)))), axis = 1)))
    G = cvxopt.matrix(G) 
    h = cvxopt.matrix(np.hstack((np.ones(c) * (-1), np.zeros(c))))     

    #solve thhe matrices equation
    svm_model = np.array(cvxopt.solvers.coneqp(P,q,G,h)['x']).flatten()
    return svm_model


SVM prediction function

In [5]:
def svm_predict_primal(data_test, label_test, svm_model):
    c, d = X_data_train.shape
    accuracy_test = accuracy_score(label_test, np.sign(np.dot(data_test, svm_model[:d]) + svm_model[-1]))
    #y1=data_test@
    return accuracy_test

Check the function with C=100, whether it is optimal or not

In [17]:
#Test the function
svm_model = svm_train_primal(X_data_train , y_label_train , 100)
test_acc_primal = svm_predict_primal(X_data_val , y_label_val, svm_model)


#print the function
print("Test accuracy : ", test_acc_primal)
print("Test accuracy in percent  : ", test_acc_primal*100 ,"%")


     pcost       dcost       gap    pres   dres
 0:  7.1760e+00  6.0141e+02  4e+04  3e+00  2e+04
 1:  2.1856e+02 -8.5822e+02  1e+03  7e-02  4e+02
 2:  1.4644e+02 -1.6271e+02  3e+02  2e-02  9e+01
 3:  5.1808e+01 -1.8561e+01  7e+01  3e-03  2e+01
 4:  2.0899e+01 -1.2897e+00  2e+01  1e-03  5e+00
 5:  1.1460e+01  3.6711e+00  8e+00  3e-04  1e+00
 6:  7.9625e+00  5.4719e+00  2e+00  7e-05  4e-01
 7:  6.9290e+00  6.0028e+00  9e-01  2e-05  9e-02
 8:  6.5689e+00  6.1933e+00  4e-01  3e-06  2e-02
 9:  6.4099e+00  6.3000e+00  1e-01  7e-07  3e-03
10:  6.3628e+00  6.3337e+00  3e-02  1e-07  6e-04
11:  6.3479e+00  6.3452e+00  3e-03  5e-09  3e-05
12:  6.3466e+00  6.3463e+00  2e-04  2e-10  1e-06
13:  6.3464e+00  6.3464e+00  5e-06  4e-12  2e-08
Optimal solution found.
Test accuracy :  0.9695555555555555
Test accuracy in percent  :  96.95555555555555 %


Equation for : 

1.   Weight
2.   Bias



In [11]:
#calculate the Weight and bias from bias form
wprimal, bprimal = svm_model[:X_data_train.shape[1]], svm_model[-1]
#series of the weight and bias
W_series, b_series = pd.Series(wprimal), pd.Series(bprimal)
#print out the answer
print("Bias is : " ,bprimal)
print("Total weight is," , np.sum(W_series))



Bias is :  1.779809171974163
Total weight is, -0.14513267953256792


**SVM Sckit-Learn check with C=100**

In [12]:
#Check the Weight , Bias , and Support Vector for Primal
SVM_classifier_1 = SVC(C = 100 / X_data_train.shape[0], kernel = 'linear')
SVM_classifier_1.fit(X_data_train, y_label_train) 
print(np.sum(SVM_classifier_1.coef_))
print(SVM_classifier_1.intercept_)
print(np.shape(SVM_classifier_1.support_vectors_))

-0.14375264622135508
[1.77953045]
(392, 200)


In [14]:

SVM_classifier = SVC(C = 100 / X_data_train.shape[0], kernel = 'linear')

SVM_classifier.fit(X_data_train, y_label_train) 
#print(SVM_classifier.fit)
y1 = SVM_classifier.predict(X_data_train)
train_accuracy = accuracy_score(y_label_train, y1)
predicted_label = SVM_classifier.predict(Y)
test_accuracy = accuracy_score(y, predicted_label)
print(' training accuracy is : ',train_accuracy)
print(' testing accuracy is : ',test_accuracy)
print(np.shape(SVM_classifier.support_vectors_))

 training accuracy is :  0.9795
 testing accuracy is :  0.968
(392, 200)
